In [4]:
import random

import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from generator import Generator
from keras import metrics
import importlib
import inverse
from matplotlib.ticker import PercentFormatter

In [5]:
def custom_loss(y_true, y_pred):
    pass

In [6]:
import tools.reportParser as parser
dataset, paths = parser.parse_all_files('data/reports/')

C:\Users\Admin\Desktop\iso\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\Admin\Desktop\iso\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(io.StringIO(value), sep="\s+|\t+|\s+\t+|\t+\s+")
C:\Users\Admin\Desktop\iso\isotherm-analyzer\tools\reportParser.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted

In [7]:
model = tf.keras.models.load_model('data/models/silica_random30K_narrow.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})
#model = tf.keras.models.load_model('data/models/silica_best_tyhanov.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})

In [8]:
cut_dataset = []
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
for i, iso in enumerate(dataset):
    if iso['p_adsorption'][0] > pressures[77] or iso['p_adsorption'][-1] < pressures[-10]:
        continue
    cut_dataset.append(iso)

In [9]:
pressures = np.load("data/initial kernels/Pressure_Silica.npy")[77:-10]
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

In [10]:
isotherms = []
for iso in cut_dataset:
    isotherms.append(np.interp(pressures, iso['p_adsorption'], iso['adsorption']))

In [11]:
prediction = model.predict(np.array(isotherms))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [12]:
import random
i = random.randint(0, len(prediction))
plt.plot(pore_widths, prediction[i], marker=".")
restored_isotherm = np.dot(kernel.T, prediction[i])
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".")
plt.plot(pressures, isotherms[i], marker="*")
plt.show()

In [13]:
def calculate_error(isotherm_real, restored_isotherm):
    defined_error = np.sum(np.abs(restored_isotherm - isotherm_real[77:-10]) / isotherm_real[77:-10]) / len(isotherm_real[77:-10])
    return defined_error


error_lst = []
for i in range(len(prediction)):
    restored_isotherm = np.dot(kernel.T, prediction[i])
    error_lst.append(calculate_error(restored_isotherm, isotherms[i])*100)

In [14]:
plt.hist(error_lst, bins=20)
plt.show()

In [15]:
model_tyhanov = tf.keras.models.load_model('data/models/silica_best_tyhanov.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss})
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
prediction_tyhanov = model_tyhanov.predict(np.array(text_x))
error_lst_tyhanov = []
for i in range(len(text_x)):
    restored_isotherm = np.dot(kernel.T, prediction_tyhanov[i])
    error_lst_tyhanov.append(calculate_error(restored_isotherm, text_x[i])*100)
error_lst_tyhanov = np.sort(error_lst_tyhanov)[:-2]

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [16]:
plt.hist(error_lst_tyhanov, bins=20)
plt.show()

In [17]:
error_lst_math = []
math_kernel = kernel[:, 77:-10]
for i in range(len(prediction)):
    math_PSD = inverse.fit_linear(isotherms[i], math_kernel, 0).x
    restored_isotherm = np.dot(kernel.T, math_PSD)
    error_lst_math.append(calculate_error(restored_isotherm, isotherms[i])*100)

In [71]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [19]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [20]:
plt.hist(error_lst_math, bins=15, alpha=0.5, label='Mathematical solution',
         weights=np.ones(len(error_lst_math)) / len(error_lst_math), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [21]:
plt.hist(error_lst, bins=15, alpha=0.5, label='Model gauss',
         weights=np.ones(len(error_lst)) / len(error_lst), range=[0, 40], edgecolor="k")
plt.hist(error_lst_tyhanov, bins=15, alpha=0.5, label='Model real PSD',
         weights=np.ones(len(error_lst_tyhanov)) / len(error_lst_tyhanov), range=[0, 40], edgecolor="k")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='upper right')
plt.ylabel("")
plt.xlabel("Fitted error")
plt.show()

In [22]:
len(isotherms)

609

In [33]:
i = 3
restored_isotherm = np.dot(kernel.T, prediction[i])
math_PSD = inverse.fit_linear(isotherms[i], math_kernel, 0).x
restored_isotherm_math = np.dot(kernel.T, math_PSD)

plt.plot(pore_widths, prediction[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, isotherms[i], marker="*", label="real")
plt.legend()
plt.show()

In [24]:
test = np.load("data/datasets/tyhanov_test.npz")
text_x = test["x_test"]
text_y = test["y_test"]
len(text_x)

92

In [46]:
i = 5
prediction_tyh = model_tyhanov.predict(text_x)
restored_isotherm = np.dot(kernel.T, prediction_tyh[i])
math_PSD = inverse.fit_linear(text_x[i], math_kernel, 0.001).x

plt.plot(pore_widths, prediction_tyh[i], marker=".", label="model")
plt.plot(pore_widths, math_PSD, marker=".", label="math")
plt.legend()
plt.show()

plt.plot(pressures, restored_isotherm[77:-10], marker=".", label="model")
plt.plot(pressures, text_x[i], marker="*", label="real")
plt.legend()
plt.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [26]:
exp_file_list = ["MCM-41", "SBA-15", "SBA-16", "MIL-101", "MIL-101_2", "DUT-49", "FDM-4", "PCN-333", "PCN-777",
                 "MIL-100"]

p_exp_list = []
n_s_exp_raw_list = []
for exp_file_name in exp_file_list:
    data = pd.read_csv(f"data/real/{exp_file_name}.txt", header=None)
    # p_exp_list.append(data.iloc[:,1].to_numpy())
    # n_s_exp_raw_list.append(data.iloc[:,3].to_numpy())
    p_exp_list.append(data.iloc[:, 1].to_numpy())
    n_s_exp_raw_list.append(data.iloc[:, 3].to_numpy())
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_list = []
for i in range(len(p_exp_list)):
    n_s_exp_list.append(np.interp(pressures, p_exp_list[i], n_s_exp_raw_list[i]))

In [27]:
n_s_exp_list[0].shape

(371,)

In [63]:
structures_prediction =  model_tyhanov.predict(np.array(n_s_exp_list))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [70]:
i = -2
print(exp_file_list[i])
plt.plot(pore_widths, structures_prediction[i], marker=".")
plt.show()

restored_isotherm = np.dot(math_kernel.T, structures_prediction[i])
plt.plot(pressures, restored_isotherm, marker=".", label="model")
plt.plot(pressures, n_s_exp_list[i], marker="*", label="real")
plt.show()

PCN-777
